In [3]:
!pip install strands-agents strands-agents-tools tika


  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 222.6 MB/s eta 0:00:00
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.9-py3-none-any.whl (78 kB)
Using cached typing_inspection-0.4.1-py3-none-any.whl (14 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 201.5 MB/s eta 0:00:00
  Attempting uninstall: botocore90m╺━━━━━━━━━━━━━━━━━━━━━ 12/26 [opentelemetry-api]
    Found existing installation: botocore 1.39.3━━━━━━━━━━━━━━ 12/26 [opentelemetry-api]
    Uninstalling botocore-1.39.3:╸━━━━━━━━━━━━━━━━━━ 14/26 [botocore]-api]
      Successfully uninstalled botocore-1.39.3━━━━━━━━━━━━━━━━ 14/26 [botocore]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26/26 [strands-agents-tools]ands-agents-tools]
ERROR: pip's dependency resolver does not currently take

In [1]:
pip freeze | grep tika

Note: you may need to restart the kernel to use updated packages.


In [1]:
from strands import Agent
from strands.models import BedrockModel
from pathlib import Path
from tika import parser

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/tika/__init__.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__('pkg_resources').declare_namespace(__name__)


In [33]:
bedrock_model = BedrockModel(
  model_id="amazon.nova-lite-v1:0", 
  region_name="us-east-1",
  temperature=0.3,
  streaming=True, # Enable/disable streaming
)

## Job description analysis

In [3]:
from pydantic import BaseModel, Field
from typing import Optional, List, Dict

class MustHaveRequirementSkills(BaseModel):
    """"""
    technical_skills: Optional[str]
    experience: Optional[str]
    qualifications: Optional[List[str]]
    core_responsibilities: Optional[List[str]]
            
class GoodToHaveRequirementSkills(BaseModel):
    """"""
    additional_skills: Optional[List[str]]
    experience: Optional[str]
    qualifications: Optional[List[str]]
    core_responsibilities: Optional[List[str]]

class AdditionalScreeningCriteria(BaseModel):
    """"""
    workpolicy_conditions: Optional[str]
    general_constrains: Optional[List[str]]
        
    
    

In [4]:
class JobDescripiton(BaseModel):
    """Schema for Job Description"""
    original_job_description: str = Field(description="The complete original job description text")
    musthave_requirementskills: List[MustHaveRequirementSkills] = Field(description="List of must have requirement skills")
    goodtohave_requirementskills: List[GoodToHaveRequirementSkills]= Field(description="List of good to have requirement skills")
    additional_screening_criteria: List[AdditionalScreeningCriteria] = Field(description = "List of additional screening Criteria")

## Parse document via Tika

In [5]:
def parse_pdf(document_path:str):
    parsed_pdf = parser.from_file(document_path)
    # saving content of pdf
    # you can also bring text only, by parsed_pdf['text'] 
    # parsed_pdf['content'] returns string 
    str_content = parsed_pdf['content'] 
    return str_content


    

In [6]:
parsed_jd= parse_pdf("data/ML- JobDescription.pdf")
parsed_resume = parse_pdf("data/resume.pdf")

### Resume Analysis

In [7]:
class CompareFields(BaseModel):
    musthave_requirementskills: Dict[str, bool] = Field(description="Dictionary of must have requirements with boolean values")
    goodtohave_requirementskills: Dict[str, bool]= Field(description="Dictionary of good to have requirements with boolean values")
    additional_screening_criteria: Dict[str, bool] = Field(description="Dictionary of additional screening requirements with boolean values")
    

In [8]:
from typing import Dict, Any
class QuantitativeResumeCheck(BaseModel):
    "Schema for Resume JD Qualitative Check"""
    requirement_match: List[CompareFields] = Field(description="Dictionary of fields with boolean values")
    
    

In [9]:
class QualitativeResumeCheck(BaseModel):
    inferred_skills: str = Field(description="Inference from candidates profile")
    project_gravity: str = Field(description="Impact of candidates projects")
    ownership_initiative: str = Field(description="Has candidate taken initative in work")
    transferability_torole:str = Field(description="How easy it be was for candidate onboarding")
    bonus_experience: str = Field(description="Matches any good to have skills from job description")
    recrruiter_summary:str = Field(description="Summary by the recruiter")
    

In [10]:
job_description_agent = Agent(
    system_prompt=(f"""
      You are an experienced resume evaluator. Extract the componenets of the resume in the following format
        
        Extract all Must Have Responsibilities:
        -Technical Skills: Essential technical skills
        -Experience: Minimum years and type of relevant experience
        -Qualifications: Mandatory degrees or certifications
        -Core Responsibilities: Key duties that are non-negotiable


        -Extract all Good-to-Have Requirements:

        -Additional Skills: Preferred technical skills
        -Extra Qualifications: Bonus education or certifications
        -Bonus Experience: Extra experience that could add value


        Extract any Additional Screening Criteria:

        -Filtering statements that affect eligibility
        -Work policy conditions
        -Availability constraints
        -Discriminatory or biased phrasing
        -Anything else that significantly affects who should or shouldn't apply

    
        """
    ), model = bedrock_model
)



In [11]:
resume_jd_agent_quantative= Agent(
system_prompt = (f"""
You are a recruiter evaluating a candidate's resume against a given job_description json inputs. Compare the candidates resume against given job description. Evaluate whether the candidate meets the necessary requirements.

Step 1: Quantitative Check
Perform a Boolean (true/false) check for each requirement based on the candidate's resume:

-For each skill listed in must_have_requirements and good_to_have_requirements of jobdescription, determine if the candidate possesses it. Return true or false for each.
-For each core_responsibility mentioned in jobdescription, determine if the candidate resume has demonstrated it in their past work. Return true or false.
-For each additional_screening_criteria in jobdescription, return a boolean value indicating whether the candidates resume meets the condition (e.g., full-time, onsite position, work authorization, etc.).

"""),
model = bedrock_model
)

In [12]:
resume_jd_agent_qualitativecheck = Agent(
system_prompt = ("""

Step 2: Qualitative Check
Now, switch to a recruiter-style qualitative assessment. Use your intuition like a human — go beyond what's explicitly stated. Read between the lines, infer intent, and use contextual clues from the resume and the JD to judge fit. Reference the results from Step 1 as part of your reasoning.

    Assess the following:

    -Inferred Skills: What skills can you infer from the candidate's projects or roles?
    -Project Gravity: Were the projects academic or real-world, high-impact, production-ready, etc.?
    -Ownership and Initiative: Did the candidate lead the work? Show initiative? Or just follow directions?
    -Transferability to Role: How well would their experience transfer to this particular role? Will they onboard quickly?
    -Bonus Experience & Extra Qualifications: If the JD lists any bonus criteria (e.g., fintech, B2B SaaS), consider that a positive signal even if not part of Step 1.
    -Recruiter Style Summary: Provide a recruiter style summary of the full profile of the candidate, consider the results of both the quantitative and qualitative assessment.

"""),model = bedrock_model
)


In [13]:
resume_finalrecommendation = Agent(
system_prompt = ("""

Step 2: Final Recommendation
Now, as a recruiter, take the final call if the candidate is the right fit for the role.Based on the summary provided. Keep the tone practitcal.


"""),model = bedrock_model
)


In [30]:
import json
def process_workflow(parsed_jd, parsed_resume):

    job_description_response = job_description_agent.structured_output(JobDescripiton, parsed_jd)
    
    # Step 1: Qualitative Resume Analysis

    append_para = [str(job_description_response), parsed_resume]
    jd_resume = "\n\n".join(append_para)

    resume_agent_quanitativecheck = resume_jd_agent_quantative(jd_resume)


    #Step 2: Qualitative Resume Analysis
    resume_agent_qualitativecheck = resume_jd_agent_qualitativecheck(str(resume_agent_quanitativecheck))
    

    # Step 3: Final Report
    final_report = resume_finalrecommendation(str(resume_agent_qualitativecheck))

    return  final_report

In [34]:
for chunk in process_workflow(parsed_jd, parsed_resume).message['content'][0]['text']:
        print(chunk, "")

<thinking> To extract the components of the resume, I need to identify the must-have responsibilities, good-to-have requirements, and additional screening criteria from the provided job description. I will categorize each requirement accordingly. </thinking>

Tool #3: JobDescripiton
Based on the provided job description and the candidate's resume, here is a quantitative evaluation of whether the candidate meets the necessary requirements:

### Must Have Requirements

#### Skills
- **Strong proficiency in Python and Java, as well as other programming languages used in machine learning.**
  - **True**: The candidate lists proficiency in Python, deep learning frameworks (PyTorch, MXNet), R, and C/C++.

- **Experience with TensorFlow, PyTorch, Keras, and scikit-learn.**
  - **Partially True**: The candidate lists proficiency in PyTorch and scikit-learn but does not mention TensorFlow or Keras.

#### Experience
- **3-5 years of experience in machine learning, data engineering, or software d

ParamValidationError: Parameter validation failed:
Invalid type for parameter messages[0].content, value: Based on the provided job description and the candidate's resume, here is a quantitative evaluation of whether the candidate meets the necessary requirements:

### Must Have Requirements

#### Skills
- **Strong proficiency in Python and Java, as well as other programming languages used in machine learning.**
  - **True**: The candidate lists proficiency in Python, deep learning frameworks (PyTorch, MXNet), R, and C/C++.

- **Experience with TensorFlow, PyTorch, Keras, and scikit-learn.**
  - **Partially True**: The candidate lists proficiency in PyTorch and scikit-learn but does not mention TensorFlow or Keras.

#### Experience
- **3-5 years of experience in machine learning, data engineering, or software development.**
  - **True**: The candidate has over 5 years of post-PhD experience in ML/AI.

#### Qualifications
- **Bachelor's degree or master's degree in computer science, dat